# MLP-CW4 Exploration Framework 


## Imports

In [1]:
# set the matplotlib backend so figures can be saved in the background
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import matplotlib
import keras
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2

import shutil
import fnmatch
import pickle
from keras import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Input
from keras.constraints import max_norm
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU
import keras.initializers
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.optimizers
from keras.utils import np_utils
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3

from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa

/home/as12production/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Environment Settings

In [2]:
#Keras setting
K.set_image_dim_ordering('tf')  #Image dimension ordering

#Random seeds
SEED = 2082018
np.random.seed(SEED)

#Image Settings
IMAGE_RESIZE = (224,224)  # Image size. Process on the by data generator. Should match CNN model input. 
IMAGE_INPUT_SIZE = (224,224,3) #Image Input size to the neural network

#Training Settings
BATCH_SIZE = 32
EPOCH = 30

#Directories
# NOTE: The labels are determined by subfolders. PNG or JPEG images only.
TRAIN_DIR = '../TRAIN' 
VAL_DIR = '../VALID'
TEST_DIR =  '../TEST'

#Index of the class label represents numerical representation
CLASS_LABELS = ["Benign", "Malignant"]
NUM_CLASSES = 2
#Checkpoints and save files

#Saving every epochs that improve val accuracy
#MODEL_CHECKPOINT_FILE="baseline_model-weights-{epoch:02d}-{val_acc:.2f}.hdf5"
# Rewriting save file for epoch that improves val accuracy
MODEL_CHECKPOINT_FILE="inception3-weights.hdf5"  

#Training charts and graphics
MODEL_TRAIN_RESULTS_FILE="inception3_train.pickle"
MODEL_ACCURACY_GRAPH_FILE="inception3_accuracy.pdf"
MODEL_LOSS_GRAPH_FILE="inception3_loss.pdf"
MODEL_EVALUATION_CM_FILE="inception3_CM.pdf"

## Loading Dataset

In [3]:
def buildImageDataset(path, imageResize=None,shuffle=False,seed=0):
    """
    Load dataset into an array. Labels are defined by folder name.
    """
    filenames = []
    data = []
    labels = []
    imagePaths = sorted(list(paths.list_images(path)))
    
    if shuffle == True:
        random.seed(seed)
        random.shuffle(imagePaths)
    
    for imagePath in imagePaths:
        image = cv2.imread(imagePath)
        # Pre-process image here
        #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        #image = clahe.apply(image)
        #image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        if imageResize != None:
            image = cv2.resize(image, imageResize)
        image = img_to_array(image)
        data.append(image)
        filenames.append(imagePath)
        label = imagePath.split(os.path.sep)[-2]
        labels.append(CLASS_LABELS.index(label))
    return (np.array(data), np.array(labels), np.array(filenames))

def showClassDistribution(y, labels):
    figure = plt.figure(figsize=(10,5))
    ax = sns.countplot(x = y)
    ax.set_xticklabels(labels)
    plt.show()

def imageResizer(dataset, imageResize):
    result = []
    for image in dataset:
        result.append(ia.imresize_single_image(image, imageResize))
    return np.array(result)

# Augmentation

In [4]:

run_this = 0

if run_this == 1:
    LR = iaa.Sequential([iaa.Fliplr(1)])
    UD = iaa.Sequential([iaa.Fliplr(1)])
    RT90 = iaa.Sequential([iaa.Affine(rotate=90)])
    RT180 = iaa.Sequential([iaa.Affine(rotate=180)])
    RT270 = iaa.Sequential([iaa.Affine(rotate=270)])



fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(X_train[-1])
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(X_train[1])
ax3 = fig.add_subplot(2,2,3)
ax3.imshow(X_train[-1])
ax4 = fig.add_subplot(2,2,4)
ax4.imshow(X_train[1])

# End augmentaiton

In [5]:
run_this = 3

# Load and save

if run_this == 3:
    X_train, y_train, train_files = buildImageDataset(TRAIN_DIR,imageResize=None,seed=SEED)
    X_valid, y_valid, valid_files = buildImageDataset(VAL_DIR,imageResize=None,seed=SEED)
    X_test, y_test, test_files = buildImageDataset(TEST_DIR,imageResize=None,seed=SEED)
    
if run_this == 1:
    X_train, y_train, train_files = buildImageDataset(TRAIN_DIR,imageResize=None,seed=SEED)
    X_valid, y_valid, valid_files = buildImageDataset(VAL_DIR,imageResize=None,seed=SEED)
    X_test, y_test, test_files = buildImageDataset(TEST_DIR,imageResize=None,seed=SEED)
    
    with open("X_train.pickle", "wb") as output_file:
            pickle.dump(X_train, output_file)

    with open("y_train.pickle", "wb") as output_file:
            pickle.dump(y_train, output_file)

    with open("X_valid.pickle", "wb") as output_file:
            pickle.dump(X_valid, output_file)

    with open("y_valid.pickle", "wb") as output_file:
            pickle.dump(y_valid, output_file)
    
    with open("X_test.pickle", "wb") as output_file:
            pickle.dump(X_valid, output_file)

    with open("y_test.pickle", "wb") as output_file:
            pickle.dump(y_valid, output_file)
            
# Quick load from saved files
if run_this == 2:
    X_train = pickle.load( open( "X_train.pickle", "rb" ) )
    y_train = pickle.load( open( "y_train.pickle", "rb" ) )
    X_valid = pickle.load( open( "X_valid.pickle", "rb" ) )
    y_valid = pickle.load( open( "y_valid.pickle", "rb" ) )
    X_test = pickle.load( open( "X_test.pickle", "rb" ) )
    y_test = pickle.load( open( "y_test.pickle", "rb" ) )



### Selecting Magnification

In [6]:
#["-40","-100-","-200-","-400-"]
magnifications = "-40-"

run_this = 1

if run_this == 1:
    validDataFrame = pd.DataFrame({'label':y_valid, 'filename':valid_files})
    trainDataFrame = pd.DataFrame({'label':y_train, 'filename':train_files})
    testDataFrame = pd.DataFrame({'label':y_test, 'filename':test_files})
    index = [i for i,item in enumerate(trainDataFrame["filename"]) if magnifications in item]
    X_train = X_train[index]
    y_train = y_train[index]
    train_files = train_files[index]
    
    index = [i for i,item in enumerate(validDataFrame["filename"]) if magnifications in item]
    X_valid = X_valid[index]
    y_valid = y_valid[index]
    valid_files = valid_files[index]
    
    index = [i for i,item in enumerate(testDataFrame["filename"]) if magnifications in item]
    X_test = X_test[index]
    y_test = y_test[index]
    test_files = test_files[index]
    
    with open("Train set " + magnifications + ".txt", 'w') as the_file:
        the_file.writelines(train_files)
        
    with open("Valid set " + magnifications + ".txt", 'w') as the_file:
        the_file.writelines(valid_files)
        
    with open("Test set " + magnifications + ".txt", 'w') as the_file:
        the_file.writelines(test_files)
      

images_aug1 = LR.augment_images(X_train)
images_aug2 = UD.augment_images(X_train)
images_aug3 = RT90.augment_images(X_train)
images_aug4 = RT180.augment_images(X_train)
images_aug5 = RT270.augment_images(X_train)
X_train = np.vstack((np.vstack((
                     np.vstack((
                         np.vstack((
                             np.vstack((images_aug1,images_aug2)),images_aug3)),images_aug4)),images_aug5)),X_train))
y_temp = np.array([])
for i in range(6):
    y_temp = np.concatenate((y_temp,y_train))
y_train = y_temp

In [8]:
print(X_train.shape)
print(y_train.shape)

(1368, 460, 700, 3)
(1368,)


## Data Exploration

### Training Set 

In [ ]:
trainDataFrame = pd.DataFrame({'label':y_train, 'filename':train_files})
print("There are {} items in training set.".format(len(y_train) ))
trainDataFrame.head(10)

In [ ]:
showClassDistribution(y_train, CLASS_LABELS)

### Validation Set

In [ ]:
validDataFrame = pd.DataFrame({'label':y_valid, 'filename':valid_files})
print("There are {} items in validation set.".format(len(y_valid) ))
validDataFrame.head(10)

In [ ]:
showClassDistribution(y_valid, CLASS_LABELS)

### Test set

In [ ]:
testDataFrame = pd.DataFrame({'label':y_test, 'filename':test_files})
print("There are {} items in test set.".format(len(y_test) ))
testDataFrame.head(10)

## Pre-Processing


In [ ]:
class ImageStandardizer:
    """
    This class standardizer image to zero mean and unit variance
    Normalization is done for each image channel
    """
    def __init__(self, eps=1e-7):
        self._mean = 0
        self._std = 0
        self._ready = False
        self._epsilon = eps # To prevent divide by zero
    
    def fit(self, train):
        self._mean = np.mean(train,axis=(0, 1, 2, 3))
        self._std = np.std(train, axis=(0, 1, 2, 3))
        self._ready = True
    
    def transform(self, data):
        assert self._ready == True, "ImageStandardizer must be initialized before use. Use fit() to initialize."
        return (data - self._mean)/(self._std + self._epsilon)
    

### Standardization

In [ ]:
run_this = 0

if run_this == 1:
    Standardizer = ImageStandardizer()
    ImageStandardizer.fit(X_train)
    X_train = ImageStandardizer.transform(X_train)
    X_valid = ImageStandardizer.transform(X_valid)

In [ ]:
X_train /= 255.0
X_valid /= 255.0
X_test /= 255.0

### Data Augmentation / Data Generator

See Keras documentation for details https://faroit.github.io/keras-docs/1.1.0/preprocessing/image/


In [ ]:
#No augmentation on baseline. Only normalize to [0,1.0] scale.
trainDataGenerator = ImageDataGenerator(rescale=1./255)
validDataGenerator = ImageDataGenerator(rescale=1./255)
testDataGenerator = ImageDataGenerator(rescale=1./255)
#  data augmentation
#trainDataGenerator = ImageDataGenerator(
#            rescale=1./255,
#            featurewise_center=False,  # set input mean to 0 over the dataset
#            samplewise_center=False,  # set each sample mean to 0
#          featurewise_std_normalization=False,  # divide inputs by std of the dataset
#           samplewise_std_normalization=False,  # divide each input by its std
#           zca_whitening=False,  # apply ZCA whitening
#           rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
#            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#            horizontal_flip=True,  # randomly flip images
#            vertical_flip=False)  # randomly flip images
 #(std, mean, and principal components if ZCA whitening is applied).

trainDataGenerator.fit(X_train)
validDataGenerator.fit(X_valid)
testDataGenerator.fit(X_test)
y_train = keras.utils.to_categorical(y_train)
y_valid = keras.utils.to_categorical(y_valid)
y_test = keras.utils.to_categorical(y_test)


# Segmentation

In [11]:
#split images into regions

def splitImage(image, n_split, resize):
    """
    Takes a numpy image matrix, and split them based into nxn matrix
    return numpy matrix (regions, rows, columns, channel)
    """
    import numpy as np
    from scipy.misc import imresize 
    #Calculate region width and height
    block_r = int(np.floor(image.shape[0] / n_split))
    block_c = int(np.floor(image.shape[1] / n_split))
    
    result = []
    #Split the image based on block_r and block_c, and append it to result array
    row = 0
    for _ in range(0,n_split-1):
        col = 0
        start_r = row  * block_r
        for _ in range(0,n_split-1):
            start_c = col  * block_c
            result.append(imresize(image[start_r:start_r + block_r, start_c:start_c + block_c,:],resize))
            col += 1
        #Some images may not divide evenly, so use the remaining pixels on the last patch
        start_c = col * block_c
        result.append(imresize(image[start_r:start_r + block_r, start_c:,:],resize))
        row += 1
    #Some images may not divide evenly, so use the remaining pixels on the last patch
    start_r = row  * block_r
    col = 0
    for _ in range(0,n_split-1):
        start_c = col  * block_c
        result.append(imresize(image[start_r:, start_c:start_c + block_c,:],resize))
        col += 1
    #Some images may not divide evenly, so use the remaining pixels on the last patch
    start_c = col * block_c
    result.append(imresize(image[start_r:, start_c:, :],resize))
    return np.array(result).astype('float')/255.0


def regionScore(image, threshold=(0.5,0.5,0.5)):
    """
    Takes a numpy image matrix (row,col,channels) and associated threshold value for each channels.
    Return a copy of matrix with anything greater than or equal to threshold sets to 
    """
    import numpy as np
    
    def color_threshold(img, th):
        """
        Takes a numpy image matrix (row,col,channels) and associated threshold value for each channel. 
        Return a copy of matrix with anything greater than or equal to threshold sets to one and zero for everything else.
        """
        assert img.shape[2] == len(th), "number of channels must be equal to number of threshold"
        result = np.copy(img)
        for channel, value in enumerate(th):
            low_values_flags = result[:,:,channel] < value  # Where values are low
            high_values_flags = result[:,:,channel] >= value  # Where values are low
            result[low_values_flags,channel] = 1  # All low values set to 0
            result[high_values_flags,channel] = 0  # All low values set to 0
        return result

    
    return np.sum(color_threshold(image, threshold))

def isRegionOfInterest(image, threshold=(0.5,0.5,0.5), perecent_threshold=0.25):
    """
    Takes a numpy image matrix (row,col,channels) and associated threshold value for each channels.
    Return a copy of matrix with anything greater than or equal to threshold sets to 
    """
    import numpy as np
    
    def color_threshold(img, th):
        """
        Takes a numpy image matrix (row,col,channels) and associated threshold value for each channel. 
        Return a copy of matrix with anything greater than or equal to threshold sets to one and zero for everything else.
        """
        assert img.shape[2] == len(th), "number of channels must be equal to number of threshold"
        result = np.copy(img)
        for channel, value in enumerate(th):
            low_values_flags = result[:,:,channel] < value  # Where values are low
            high_values_flags = result[:,:,channel] >= value  # Where values are low
            result[low_values_flags,channel] = 1  # All low values set to 0
            result[high_values_flags,channel] = 0  # All low values set to 0
        return result

    
    if ( float(np.sum(color_threshold(image, threshold))) / totalImageSize(image)) > float(perecent_threshold):
        return True
    else:
        return False

def segmentCell2(set_of_images, resize):
    cell = []
    for index, image in enumerate(set_of_images):
        for patch in splitImage(image,6,resize):
            if (isRegionOfInterest(patch, threshold=(0.5,0.5,0.5), perecent_threshold=0.20)):
                cell.append(patch)
    return np.array(cell)

def segmentCell(set_of_images, resize):
    cell = []
    for index, image in enumerate(set_of_images):
        score = []
        patches = []
        for patch in splitImage(image,6,resize):
            score.append(regionScore(patch, threshold=(0.5,0.5,0.5)))
            patches.append(patch)
        # Non-Maximum suppression
        score = np.array(score)
        patches = np.array(patches)
        cell.append(patches[np.argmax(score)])
    return np.array(cell)

In [12]:
X_tr = segmentCell(X_train, IMAGE_RESIZE)
X_vl = segmentCell(X_valid, IMAGE_RESIZE)

/home/as12production/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/as12production/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/as12production/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/as12production/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


MemoryError: 

In [ ]:
print(X_tr.shape)
print(X_train.shape)

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_valid = keras.utils.to_categorical(y_valid)
X_train = X_tr
X_valid = X_vl

In [ ]:
num = 5
for i in range(num):
    plt.imshow(X_train[i]*255)
    plt.show()

# Building Neural Net Model

In [ ]:
# Building Models
def InceptionNet(verbose=False):
    #https://keras.io/applications/#inceptionv3
    #Use Inception 3 without the last layer.
    #Replace last layer with 1 sigmoid for binary classification
    sgd = SGD(lr=0.01, momentum=0.9,nesterov=False)
    model = keras.applications.inception_v3.InceptionV3(include_top=False,
                                                        weights='imagenet',  #Pre-train on ImageNet 
                                                        input_tensor=Input(shape=IMAGE_INPUT_SIZE),
                                                        input_shape=None,
                                                        pooling='avg',
                                                        classes=NUM_CLASSES)
    final = Model(input=model.input,output=Dense(NUM_CLASSES, activation='softmax')(model.output))
    if verbose == True:
        final.summary()
    final.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])    
    return final

model = InceptionNet()

#### Setting checkpoint options

In [ ]:
checkpoint = ModelCheckpoint(MODEL_CHECKPOINT_FILE, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#### Loading weights

In case issue with training / crash. Run this to load the weights into model. You can either train the model or use it for classification.


In [ ]:
run_this = 0

#Specify checkpoint file here
CHECKPOINT_FILE = MODEL_CHECKPOINT_FILE

###############################################
if run_this == 1:
    model.load_weights(CHECKPOINT_FILE)

# Training W/O Generator

In [ ]:
run_this = 1

if run_this == 1:
    #Train models
   
    history = model.fit(x=X_train,y=y_train,
                        validation_data=(X_valid,y_valid),
                        batch_size=BATCH_SIZE,
                        epochs = EPOCH,
                       callbacks = callbacks_list)
    
    #Saving training result
    with open(MODEL_TRAIN_RESULTS_FILE, "wb") as output_file:
        pickle.dump(history.history, output_file)

# Training w/ Generator

In [ ]:
run_this = 0

if run_this == 1:
    #Train models
    history = model.fit_generator(
        trainDataGenerator.flow(X_train, y_train),
        epochs = EPOCH,
        steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
        validation_data=validDataGenerator.flow(X_valid, y_valid),
        callbacks = callbacks_list)
    
    #Saving training result
    with open(MODEL_TRAIN_RESULTS_FILE, "wb") as output_file:
        pickle.dump(history.history, output_file)

#### Visualizing Training

In [ ]:
run_this = 1

if run_this == 1:
    with open(MODEL_TRAIN_RESULTS_FILE, "rb") as input_file:
        history = pickle.load(input_file)
    plt.style.use('ggplot')
    accuracy_plot = plt.figure(figsize=(15,10))
    for k in ['val_acc', 'acc']:
        data = np.array(history[k])
        plt.plot(data)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch Number')
    plt.legend(['acc(valid)', 'acc(train)'], loc='upper left')
    plt.grid(True)
    plt.show()
    loss_plot = plt.figure(figsize=(15,10))
    for k in ['loss', 'val_loss']:
        data = np.array(history[k])
        plt.plot(data)
    plt.title('Model Loss')
    plt.ylabel('Error (Log Loss)')
    plt.xlabel('Epoch Number')
    plt.grid(True)
    plt.legend(['error(train)', 'error(valid)'], loc='upper left')
    plt.show()
    #Save visualization data
    print("Val Acc: ",np.max(np.array(history['val_acc'])))
    print("Train Acc: ", np.max(np.array(history['acc'])))
    print("Val Err: ",np.max(np.array(history['val_loss'])))
    print("Train Err: ", np.max(np.array(history['loss'])))
    accuracy_plot.savefig(MODEL_ACCURACY_GRAPH_FILE, bbox_inches='tight')
    loss_plot.savefig(MODEL_LOSS_GRAPH_FILE, bbox_inches='tight')

# Evaluation

## Validation Set Confusion Matrix

#### Note: 
The labels are hard coded and might not represent the actual label as automatically created by DataGenerator. So switch it around until we figure out the solution

In [ ]:
run_this = 1


    
def plot_confusion_matrix(cm, classes=None, title='Confusion matrix'):
    """Plots a confusion matrix."""
    plot = plt.figure()
    if classes is not None:
        sns.heatmap(cm, xticklabels=classes, yticklabels=classes, vmin=0., vmax=1., annot=True)
    else:
        sns.heatmap(cm, vmin=0., vmax=1.)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    return plot
    
    
if run_this == 1:
    #model.load_weights(MODEL_CHECKPOINT_FILE)
    y_pred = model.predict(X_valid)
    cm = confusion_matrix(np.argmax(y_valid,axis=1), np.argmax(np.rint(y_pred),axis=1))
    cm_norm = cm/cm.sum(axis=1)[:, np.newaxis]
    plot = plot_confusion_matrix(cm_norm, classes=CLASS_LABELS)
    plot.savefig(MODEL_EVALUATION_CM_FILE, bbox_inches='tight')

In [ ]:
y_pred


## Test Data
Scalar test loss (if the model has no metrics) or list of scalars (if the model computes other metrics). The attribute model.metrics_names will give you the display labels for the scalar outputs.

In [ ]:
run_this = 1

if run_this == 1:
    scores = model.evaluate(X_test,y_test)
    print(scores)
    print(model.metrics_names)
    

In [ ]:
max(y_pred)

In [ ]:
run_this = 1
if run_this == 1:
    scores = model.evaluate(X_valid,y_valid)
    print(scores)
    print(model.metrics_names)

In [ ]:
model2 = InceptionNet()
model2.load_weights(MODEL_CHECKPOINT_FILE)

In [ ]:
if run_this == 1:
    scores = model2.evaluate(X_valid,y_valid)
    print(scores)
    print(model2.metrics_names)

In [ ]:
if run_this == 1:
    #model.load_weights(MODEL_CHECKPOINT_FILE)
    y_pred = model2.predict(X_valid)
    cm = confusion_matrix(np.argmax(y_valid,axis=1), np.argmax(np.rint(y_pred),axis=1))
    cm_norm = cm/cm.sum(axis=1)[:, np.newaxis]
    plot = plot_confusion_matrix(cm_norm, classes=CLASS_LABELS)
    plot.savefig(MODEL_EVALUATION_CM_FILE, bbox_inches='tight')